### Descripción del código

Este código analiza la nubosidad observada en superficie para distintas estaciones meteorologicas

Los datos de nubosidad vienen dadas en oktas y tienen que estar guardados en *ruta_a_archivos* (un archivo por estacion)

Se hace lo siguiente: 

**Cálculo de frecuencias relativas**
* Se calcula la frecuencia relativa mensual para todos los meses que cuenten con mas del 80% de datos no faltantes. Esta serie es la que se usa para hacer los siguientes calculos

**Climatología:** 
* Climatología de frecuencia relativa mensual. Se usa el período comun con el satelite para calcularlo: Diciembre 1983 a Noviembre 2016. 
* Esta climatologia ademas la agrupo: sumo las frecuencias relativas medias para cada mes de las oktas de un grupo, los grupos son: 0, 1-2-3, 4, 5-6-7, 8
* Se grafica la climatologia con graficos de barras y stackedplots

**Climatología: media espacial**
* Climatología de frecuencia relativa mensual. Se usa el período comun con el satelite para calcularlo: Diciembre 1983 a Noviembre 2016. 
* Se calcula el promedio espacialmente
* Esta climatologia ademas la agrupo: sumo las frecuencias relativas medias para cada mes de las oktas de un grupo, los grupos son: 0, 1-2-3, 4, 5-6-7, 8
* Se grafica la climatologia con graficos de barras y stackedplots

**Cálculo anomalias:**
* A cada valor mensual de frecuencia de nubosidad, le resto la climatología del respectivo mes
* Calculo el promedio movil de 1 año y 5 años 
* Calculo la tendencia de Theil-Sen y la testeo con un test de Mann Kendall con un nivel de confianza del 95%

**Hago correlogramas de las series de anomalías mensuales de la frecuencia de las octas 0 y 8**

**Cálculo anomalias estacioneles:**
* Promedio las anomalias mensuales calculadas previamente por cada estacion DJF, MAM, JJA, SON
* Las grafico junto con su tendencia de Theil-Sen y la testeo con un test de Mann Kendall con un nivel de confianza del 95%

**Armo tablas con las tendencias de Theil-Sen calculadas previamente**
* Anualmente y por estaciones con las oktas agrupadas


----------------- 

**Paso 1** 

*Cargo librerias y funciones necesarias*

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from f_abrir_datos_smn import abrir_datos_smn
from f_graficos_smn import scatter, histograma, stackplot, barplot_oktas_separadas, barplot_oktas_juntas, lineplot_oktas_separadas, lineplot_oktas_juntas, histogramas_partidos, histogramas_partidos_poster 
#from f_procesamiento_smn import media_mensual, frecuencia_oktas_por_anio, frecuencia_oktas_por_anio_estaciones,frecuencia_oktas_por_mes, agrupo_frecuencias, agrupo_data
from f_procesamiento_smn import frecuencia_oktas_por_mes, agrupo_frecuencias
from f_procesamiento_nubosidad_observada_anomalias import f_tabla_tendencias_agrupo, f_tabla_tendencias_agrupo_por_estacion
from f_graficos_nubosidad_observada_anomalias import stacked_plot_barras_climatologia, stacked_plot_barras_climatologia_media_espacial, barplot_oktas_separadas_climatologia, barplot_oktas_separadas_climatologia_media_espacial, barplot_oktas_separadas_RELATIVO, barplot_oktas_separadas_RELATIVO_por_estacion

from c_clases_smn import dataframe_smn, frecuencias_oktas
import f_paletas_colores


Defino la *ruta_a_archivos* y a *ruta_a_resultados*

In [ ]:
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

trabajo_en_mi_pc = False
trabajo_en_vegeta = True

if trabajo_en_mi_pc:
    #si estoy trabajando desde mi pc parada en directorio ./Doctorado/codigos/codigos2021/nubosidad
    ruta_a_archivos = "../../../datos/smn/smn_variables_1961-2021/" #ruta donde estan los archivos con datos del smn
    ruta_a_resultados = "../../../resultados/resultados2021/nubosidad/" #ruta donde se guardan los resultados del analisis de nubosidad
     
if trabajo_en_vegeta:
    #si estoy trabajando desde el servidor vegeta
    ruta_a_archivos = "/pikachu/datos/nadia.testani/Doctorado/datos/smn/smn_variables_1961-2021/" #ruta donde estan los archivos con datos del smn
    ruta_a_resultados = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/" #ruta donde se guardan los resultados del analisis de nubosidad

print("--------------------")
print(f"La ruta a archivos es:{ruta_a_archivos}")
print("--------------------")
print(f"La ruta a resultados es: {ruta_a_resultados}")

----------------- 

**Paso 2** 

*Cargo los datos*

In [ ]:
nombres_archivos = os.listdir(ruta_a_archivos)
data = {}
for nombre_archivo in nombres_archivos:
    id_data = abrir_datos_smn(ruta_a_archivos = ruta_a_archivos, nombre_archivo = nombre_archivo, 
                              variable = "nub")
    data[id_data[0]] = id_data[1]

ides_omm = list(data.keys()) 

*Defino diccionario con numeros y nombres de estaciones ordenadas por region, extraigo ides y nombres*

In [ ]:
#cambio ides_omm para que queden ordenadas por region
estaciones_ordenadas = {87148: "Roque Saenz Peña", 87166: "Corrientes", 87178: "Posadas Aero",
                        87187: "Oberá", 87270: "Reconquista Aero", 87289: "Paso de los Libres Aero",
                        87393: "Montecaseros", 87395: "Concordia Aero"}

ides_omm_ordenado = list(estaciones_ordenadas.keys())
nombres_ides_omm_ordenado = list(estaciones_ordenadas.values())

*Defino la fecha de inicio en el que quiero analizar las series temporales de nubosidad*

In [ ]:
periodo_satelite = True
periodo_completo = False

if periodo_satelite: #periodo comun con satelite (Diciembre 1983 a Noviembre 2016)
    fecha_inicio_str = "1983-12-15"
    fecha_final_str = "2016-11-15"

if periodo_completo: #periodo completo años enteros (1961-01-01 a 2020-12-31) 
    fecha_inicio_str = "1961-01-01" 
    fecha_final_str = "2020-12-31" 

#climatologia: periodo comun con satelite (Diciembre 1983 a Noviembre 2016)
fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15"


*Defino el porcentaje de na admitido para estimar la frecuencia relativa mensual*

In [ ]:
porcentaje_na_admitido = 20

*Genero (o me paro si es que ya existe) en el directorio de resultados especifico para guardar lo que se genere*

In [ ]:
print("-----------------")
print("Manejo de directorios")
print("-----------------")
print(f"Se para en el directorio de resultados generales llamado: {ruta_a_resultados}")
os.chdir(ruta_a_resultados) #se para en ruta general de resultados de nubosidad
print("-----------------")

if "nubosidad_observada" not in os.listdir():
    print(f"Genera directorio llamado: nubosidad_observada en {ruta_a_resultados}")
    os.mkdir("nubosidad_observada")
print(f"Se para en el directorio llamado: {ruta_a_resultados}nubosidad_observada")
os.chdir("./nubosidad_observada") #se para en nubosidad_observada
print("-----------------")

if "smn" not in os.listdir():
    print(f"genera directorio llamado: smn en {ruta_a_resultados}nubosidad_observada")
    os.mkdir("smn")
print(f"Se para en el directorio llamado: {ruta_a_resultados}nubosidad_observada/smn")
os.chdir("./smn") #se para en smn
print("-----------------")

if "data_mensual" not in os.listdir():
    print(f"Genera directorio llamado: data_mensual en {ruta_a_resultados}nubosidad_observada/smn")
    os.mkdir("data_mensual")
print(f"Se para en el directorio llamado: {ruta_a_resultados}nubosidad_observada/smn/data_mensual")
os.chdir("./data_mensual") #se para en data_mensual
print("-----------------")

if f"admite_{porcentaje_na_admitido}porc_na" not in os.listdir():
    print(f"genera directorio llamado: admite_{porcentaje_na_admitido}porc_na en {ruta_a_resultados}nubosidad_observada/smn/data_mensual")
    os.mkdir(f"admite_{porcentaje_na_admitido}porc_na")
print(f"Se para en el directorio llamado: {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na")
directorio_na = f"admite_{porcentaje_na_admitido}porc_na"
os.chdir("./"+directorio_na) #se para en admite_{porcentaje_na_admitido}porc_na
print("-----------------")

if "series_diario" not in os.listdir():
    print(f"Genera directorio llamado: series_diario en {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na")
    os.mkdir("series_diario")
print(f"Se para en el directorio llamado: {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na/series_diario")
os.chdir("./series_diario") #se para en series_diario, aca se van a guardar los resultados 
print("-----------------")

----------------- 

**Paso 3** 

*Se mueve un directorio para atras, genero o me paro en carpeta datos, y genero y guardo (o abro, si ya existe) un csv con datos de frecuencia de cada okta por mes (cantidad de dias por mes con cantidad de oktas) y frecuencia relativa de cada okta por mes (cantidad de dias por mes con cantidad de oktas dividido la cantidad de dias al mes)*

*Para calcular las frecuencias por mes usamos la funcion frecuencia_oktas_por_mes de f_procesamiento_smn*

*La frecuencia se calcula sobre los meses que tienen menos del 20% de dias del mes con datos faltantes*

*Los archivos con las frecuencias los acomodo en diccionario*

In [ ]:
os.chdir("../") #se mueve uno atras: a admite_porc_na
print("-----------------") 
if "datos" not in os.listdir():
    print(f"genera directorio llamado: datos en {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na")
    os.mkdir("datos")
print(f"se para en el directorio llamado: {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na/datos")
os.chdir("./datos") #se para en datos, aca se van a guardar los resultados 
print("-----------------")    

data_frecuencias_mensuales = {} #genero diccionario donde se van a guardar las frecuencias mensuales
data_frecuencias_relativas_mensuales = {} #genero diccionario donde se van a guardar las frecuencias relativas mensuales

#para cada id_omm_ordenado
for id_omm in ides_omm_ordenado:
    ##abro o genero archivo con frecuencias
    if f"frecuencia_mensual_en_dias_{id_omm}.csv" not in os.listdir():
        print(f"genera un archivo: frecuencia_mensual_en_dias_{id_omm}.csv en {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na/datos")
        frecuencia_mensual = frecuencia_oktas_por_mes(id_omm, data[id_omm], ".")[0]
    else:
        frecuencia_mensual = pd.read_csv(f"frecuencia_mensual_en_dias_{id_omm}.csv", index_col="fecha")

    ##abro o genero archivo con frecuencias relativas
    if f"frecuencia_relativa_mensual_en_dias_{id_omm}.csv" not in os.listdir():
        print(f"genera un archivo: frecuencia_relativa_mensual_en_dias_{id_omm}.csv en {ruta_a_resultados}nubosidad_observada/smn/data_mensual/admite_{porcentaje_na_admitido}porc_na/datos")
        frecuencia_relativa_mensual = frecuencia_oktas_por_mes(id_omm, data[id_omm], ".")[1]
    else:
        frecuencia_relativa_mensual = pd.read_csv(f"frecuencia_relativa_mensual_en_dias_{id_omm}.csv", index_col="fecha")
    
    #lo acomodo en diccionarios
    data_frecuencias_mensuales[id_omm] = frecuencia_mensual
    data_frecuencias_relativas_mensuales[id_omm] = frecuencia_relativa_mensual

print("Se generaron diccionarios con data frames frecuencias mensuales y de frecuencias relativas mensuales de cada okta para cada estacion")
print("Cada elemento de esos diccionarios (un elemento por estacion) tiene esta pinta:")
print("-----------------") 
print("Frecuencias mensuales:")
print(data_frecuencias_mensuales[87166].iloc[0:3])
print("-----------------") 
print("Frecuencias relativas mensuales:")
print(data_frecuencias_relativas_mensuales[87166].iloc[0:3])
print("-----------------") 

*Vuelve a ruta con resultados especificos*

In [ ]:
#vuelvo a directorio anterior
directorio_raiz = "../series_diario"
os.chdir(directorio_raiz)
print("-----------------") 
print(f"Vuelvo a un directorio anterior. Ahora el directorio de trabajo es: {os.getcwd()}") 
print("-----------------") 

--------------------

**Paso 4: climatología calculo**

*Calculo media mensual de frecuencias relativas mensuales en el periodo climatologico*

*Agrupo la climatologia en oktas: sumo las frecuencias relativas medias para cada mes de las oktas de un grupo. Para eso uso la funcion agrupo_frecuencias de f_procesamiento_smn*

*Los grupos son: 0, 1-2-3, 4, 5-6-7, 8*

In [ ]:
climatologia_frecuencias_relativas_mensuales = {} #genero diccionario donde se van a guardar las climatologias de frecuencias relativas mensuales
climatologia_frecuencias_relativas_mensuales_agrupadas = {} #genero diccionario donde se van a guardar las climatologias de frecuencias relativas mensuales agrupadas

#para cada id_omm_ordenado
for id_omm in ides_omm_ordenado:
    #calculo climatologia
    frecuencia_relativa_mensual = data_frecuencias_relativas_mensuales[id_omm]
    frecuencia_relativa_mensual.index = pd.to_datetime(frecuencia_relativa_mensual.index)
    frecuencia_relativa_mensual_fechas_climatologia = frecuencia_relativa_mensual.loc[fecha_inicio_climatologia_str:fecha_final_climatologia_str]
    frecuencia_relativa_mensual_climatologia = frecuencia_relativa_mensual_fechas_climatologia.groupby(frecuencia_relativa_mensual_fechas_climatologia.index.month).mean() 
    frecuencia_relativa_mensual_climatologia
    #Lo guardo en diccionario: climatologia_frecuencias_relativas_mensuales
    climatologia_frecuencias_relativas_mensuales[id_omm] = frecuencia_relativa_mensual_climatologia
    #Si sumo la frecuencia media de cada okta en cada mes suma suma 100%. Es decir: tomo enero y sumo la frecuencia relativa de cada okta. Y así *
    for i in range(0,12):
        if sum(frecuencia_relativa_mensual_climatologia.iloc[i]) > 98: continue
        else: print(f"Para la estacion: {id_omm}. La suma de las frecuencias en el mes {i+1}no es mayor a 98")
    #Agrupo frecuencias
    #Lo guardo en diccionario: climatologia_frecuencias_relativas_mensuales_agrupadas
    climatologia_frecuencias_relativas_mensuales_agrupadas[id_omm] = agrupo_frecuencias(frecuencia_relativa_mensual_climatologia, grupo = 3)
    

--------------------

**Paso 4b: climatología calculo media espacial**

In [ ]:
climatologia_frecuencias_relativas_mensuales_media_espacial = (climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[0]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[1]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[2]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[3]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[4]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[5]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[6]]+climatologia_frecuencias_relativas_mensuales[ides_omm_ordenado[7]])/8

--------------------

**Paso 4: climatología graficos**

In [ ]:
#hago graficos en español
for id_omm in ides_omm_ordenado:
    stacked_plot_barras_climatologia(frecuencias = climatologia_frecuencias_relativas_mensuales[id_omm], 
                                     estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                     colores = f_paletas_colores.paleta_9_colores, agrupo = None,
                                     anio_inicio = fecha_inicio_climatologia_str[0:4], 
                                     anio_final = fecha_final_climatologia_str[0:4],
                                     idioma = "español")
    stacked_plot_barras_climatologia(frecuencias = climatologia_frecuencias_relativas_mensuales_agrupadas[id_omm], 
                                     estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                     colores = f_paletas_colores.paleta_5_colores, agrupo = 3,
                                     anio_inicio = fecha_inicio_climatologia_str[0:4], 
                                     anio_final = fecha_final_climatologia_str[0:4],
                                     idioma = "español")
    barplot_oktas_separadas_climatologia(frecuencias = climatologia_frecuencias_relativas_mensuales[id_omm],
                                         estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                         colores = f_paletas_colores.paleta_9_colores,
                                         anio_inicio = fecha_inicio_climatologia_str[0:4], 
                                         anio_final = fecha_final_climatologia_str[0:4],
                                         idioma = "español")



**PAPER:**

In [ ]:
#hago graficos medios en ingles - PAPER
stacked_plot_barras_climatologia_media_espacial(frecuencias = climatologia_frecuencias_relativas_mensuales_media_espacial, 
                                 colores = f_paletas_colores.paleta_9_colores, agrupo = None,
                                 anio_inicio = fecha_inicio_climatologia_str[0:4], 
                                 anio_final = fecha_final_climatologia_str[0:4],
                                 idioma = "ingles", titulo = False)

barplot_oktas_separadas_climatologia_media_espacial(frecuencias = climatologia_frecuencias_relativas_mensuales_media_espacial,
                                     colores = f_paletas_colores.paleta_9_colores,
                                     anio_inicio = fecha_inicio_climatologia_str[0:4], 
                                     anio_final = fecha_final_climatologia_str[0:4],
                                     idioma = "ingles", titulo = False)


--------------------

**Paso 5: calculo de anomalias**

*A cada valor mensual de frecuencia de nubosidad, le resto la climatología del respectivo mes*

In [ ]:
from f_procesamiento_nubosidad_observada_anomalias import f_serie_anomalias_mensuales, promedio_movil, tendencia, tendencia_theil_sen

In [ ]:
frecuencias_relativas_anomalias = {}
for id_omm in ides_omm_ordenado:
    frecuencia_relativa_mensual = data_frecuencias_relativas_mensuales[id_omm]
    frecuencia_relativa_mensual_climatologia = climatologia_frecuencias_relativas_mensuales[id_omm]
    frecuencia_relativa_anomalia = f_serie_anomalias_mensuales(frecuencia_relativa_mensual, frecuencia_relativa_mensual_climatologia)
    frecuencias_relativas_anomalias[id_omm] = frecuencia_relativa_anomalia

*Calculo el promedio movil de 12 meses*

In [ ]:
frecuencias_relativas_anomalias_filtro_12_meses = {}
for id_omm in ides_omm_ordenado:
    ventana = 12
    frecuencia_relativa_anomalia_media_movil_12_dic = {}
    frecuencia_relativa_anomalia = frecuencias_relativas_anomalias[id_omm]
    for i in range(0, 9):
        frecuencia_relativa_anomalia_media_movil_12_dic[str(i)] = promedio_movil(frecuencia_relativa_anomalia[str(i)], ventana)

    frecuencia_relativa_anomalia_media_movil_12 = pd.DataFrame(frecuencia_relativa_anomalia_media_movil_12_dic)
    frecuencia_relativa_anomalia_media_movil_12.index = frecuencia_relativa_anomalia.index
    frecuencias_relativas_anomalias_filtro_12_meses[id_omm] = frecuencia_relativa_anomalia_media_movil_12



*Calculo el promedio movil de 5 años*

In [ ]:
frecuencias_relativas_anomalias_filtro_12_5_meses = {}
for id_omm in ides_omm_ordenado:
    ventana_5 = 12*5
    frecuencia_relativa_anomalia_media_movil_12_5_dic = {}
    frecuencia_relativa_anomalia = frecuencias_relativas_anomalias[id_omm]
    for i in range(0, 9):
        frecuencia_relativa_anomalia_media_movil_12_5_dic[str(i)] = promedio_movil(frecuencia_relativa_anomalia[str(i)], ventana_5)

    frecuencia_relativa_anomalia_media_movil_12_5 = pd.DataFrame(frecuencia_relativa_anomalia_media_movil_12_5_dic)
    frecuencia_relativa_anomalia_media_movil_12_5.index = frecuencia_relativa_anomalia.index
    frecuencias_relativas_anomalias_filtro_12_5_meses[id_omm] = frecuencia_relativa_anomalia_media_movil_12_5


*La tendencia se calcula directamente en la funcion del graficado* 

--------------------

**Paso 4: anomalías graficos**

*Grafico para el periodo de comun con satelite:*

In [ ]:
periodo_satelite == True
periodo_completo == False

if periodo_satelite: #periodo comun con satelite (Diciembre 1983 a Noviembre 2016)
    fecha_inicio_str = "1983-12-15"
    fecha_final_str = "2016-11-15"

if periodo_completo: #periodo completo años enteros (1961-01-01 a 2020-12-31) 
    fecha_inicio_str = "1961-01-01" 
    fecha_final_str = "2020-12-31" 

#hago graficos en español
for id_omm in ides_omm_ordenado:
    barplot_oktas_separadas_RELATIVO(frecuencias = frecuencias_relativas_anomalias[id_omm],
                                     frecuencias_media_movil_12 = frecuencias_relativas_anomalias_filtro_12_meses[id_omm],
                                     frecuencias_media_movil_12_5 = frecuencias_relativas_anomalias_filtro_12_5_meses[id_omm],
                                     estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                     fecha_inicio_str = fecha_inicio_str, 
                                     fecha_final_str = fecha_final_str, 
                                     colores = f_paletas_colores.paleta_9_colores,
                                     incluir_tendencia_ts = True,
                                     incluir_media_movil_12 = True,
                                     incluir_media_movil_12_5 = False,
                                     idioma = "español")


*Grafico para el periodo completo:*

In [ ]:
periodo_satelite == False
periodo_completo == True

fecha_inicio_str = "1961-01-01" 
fecha_final_str = "2020-12-31" 

#hago graficos en español
for id_omm in ides_omm_ordenado:
    barplot_oktas_separadas_RELATIVO(frecuencias = frecuencias_relativas_anomalias[id_omm],
                                     frecuencias_media_movil_12 = frecuencias_relativas_anomalias_filtro_12_meses[id_omm],
                                     frecuencias_media_movil_12_5 = frecuencias_relativas_anomalias_filtro_12_5_meses[id_omm],
                                     estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                     fecha_inicio_str = fecha_inicio_str, 
                                     fecha_final_str = fecha_final_str, 
                                     colores = f_paletas_colores.paleta_9_colores,
                                     incluir_tendencia_ts = True,
                                     incluir_media_movil_12 = True,
                                     incluir_media_movil_12_5 = False,
                                     idioma = "español")

--------------------

**Paso 5: Correlograma con anomalias de frecuencia de octa 0 y octa 8**

*Obs: esta parte del código se puede perfeccionar*

In [ ]:
periodo_satelite == False
periodo_completo == True

fecha_inicio_str = "1961-01-01" 
fecha_final_str = "2020-12-31" 

data_frame_anomalias_frecuencias_0 = pd.concat([frecuencias_relativas_anomalias[87148]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87166]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87178]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87187]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87270]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87289]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87393]["0"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87395]["0"].loc[fecha_inicio_str:fecha_final_str],
                                             ], axis=1)
data_frame_anomalias_frecuencias_0.columns = ides_omm_ordenado

data_frame_anomalias_frecuencias_8 = pd.concat([frecuencias_relativas_anomalias[87148]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87166]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87178]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87187]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87270]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87289]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87393]["8"].loc[fecha_inicio_str:fecha_final_str],
                                              frecuencias_relativas_anomalias[87395]["8"].loc[fecha_inicio_str:fecha_final_str],
                                             ], axis=1)
data_frame_anomalias_frecuencias_8.columns = ides_omm_ordenado

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

anio_inicio = fecha_inicio_str[0:4]
anio_final = fecha_final_str[0:4]
idioma = "español"

g = sns.pairplot(data_frame_anomalias_frecuencias_0, kind = "reg", corner = False, dropna = True,
            plot_kws = dict(scatter_kws = dict(alpha=0.5, color = "red"), line_kws = dict(color="darkred")),
            diag_kws = dict(color = "red"))
if idioma == "ingles":
    g.fig.suptitle(f"Correlogram: Anomaly of daily mean total cloud cover (oktas) monthly frequency ({anio_inicio}-{anio_final}) SMN \n Okta 0", y = 1.04, fontsize = 24) # y= some height>1
elif idioma == "español":
    g.fig.suptitle(f"Correlograma: Anomalía de la frecuencia mensual de cobertura nubosa media diaria ({anio_inicio}-{anio_final}) SMN \n Octa 0", y = 1.04,fontsize = 24)

plt.show()

g = sns.pairplot(data_frame_anomalias_frecuencias_8, kind = "reg", corner = False, dropna = True,
            plot_kws = dict(scatter_kws = dict(alpha=0.5, color = "red"), line_kws = dict(color="darkred")),
            diag_kws = dict(color = "red"))
if idioma == "ingles":
    g.fig.suptitle(f"Correlogram: Anomaly of daily mean total cloud cover (oktas) monthly frequency ({anio_inicio}-{anio_final}) SMN \n Okta 8", y = 1.04, fontsize = 24) # y= some height>1
elif idioma == "español":
    g.fig.suptitle(f"Correlograma: Anomalía de la frecuencia mensual de cobertura nubosa media diaria ({anio_inicio}-{anio_final}) SMN \n Octa 8", y = 1.04,fontsize = 24)

plt.show()

--------------------

**Paso 6: Cálculo de anomalias estacionales**

*Promedio las anomalias mensuales calculadas previamente por cada estacion DJF, MAM, JJA, SON*

In [ ]:
freqs_relativas_anomalias_estacionales_todas_las_estaciones = {}

for id_omm in ides_omm_ordenado:
    freqs_relativas_anomalias_estacionales = {}
    df_data = frecuencias_relativas_anomalias[id_omm]

    for estacion in ["DJF", "MAM", "JJA", "SON"]:
        freq_r_0 = []
        freq_r_1 = []
        freq_r_2 = []
        freq_r_3 = []
        freq_r_4 = []
        freq_r_5 = []
        freq_r_6 = []
        freq_r_7 = []
        freq_r_8 = []
        freqs_relativas_anomalias_estacional = [freq_r_0, freq_r_1, freq_r_2, freq_r_3, freq_r_4, freq_r_5, freq_r_6, freq_r_7, freq_r_8]

        for anio in df_data.index.year.unique():#iterar sobre los anios

                data = df_data.loc[str(anio)]

                if estacion == "DJF":
                    if anio == df_data.index.year[0]:
                        data_estacion = np.nan
                    else:
                        data_anio_previo = df_data.loc[str(anio-1)]
                        data_estacion = pd.concat([data_anio_previo[data_anio_previo.index.month == 12],
                                                   data[data.index.month ==1],data[data.index.month == 2]])
                        media_estacion = data_estacion.mean(axis = 0)

                elif estacion == "MAM":
                    data_estacion = pd.concat([data[data.index.month == 3],data[data.index.month == 4],data[data.index.month == 5]])
                    media_estacion = data_estacion.mean(axis = 0)

                elif estacion == "JJA":
                    data_estacion = pd.concat([data[data.index.month == 6],data[data.index.month == 7],data[data.index.month == 8]])
                    media_estacion = data_estacion.mean(axis = 0)

                elif estacion == "SON":
                    data_estacion = pd.concat([data[data.index.month == 9],data[data.index.month == 10],data[data.index.month == 11]])
                    media_estacion = data_estacion.mean(axis = 0)

                if type(data_estacion) == pd.DataFrame :
                    for i in range(0,9):
                        freqs_relativas_anomalias_estacional[i].append(media_estacion.loc[str(i)])

        if estacion == "DJF":
            indices = df_data.index.year.unique()[1: len(df_data.index.year.unique())] #se saca el primer año
        else:
            indices = df_data.index.year.unique()


        freqs_relativas_anomalias_estacional_df = pd.DataFrame(freqs_relativas_anomalias_estacional).T
        freqs_relativas_anomalias_estacional_df.index = indices
        freqs_relativas_anomalias_estacionales[estacion] = freqs_relativas_anomalias_estacional_df
    freqs_relativas_anomalias_estacionales_todas_las_estaciones[id_omm] = freqs_relativas_anomalias_estacionales

--------------------

**Paso 7: Gráfico de anomalias estacionales**

*Periodo completo*

In [ ]:
periodo_satelite == False
periodo_completo == True

fecha_inicio_str = "1961-01-01" 
fecha_final_str = "2020-12-31"

for id_omm in ides_omm_ordenado:
    for estacion in ["DJF", "MAM", "JJA", "SON"]:
        barplot_oktas_separadas_RELATIVO_por_estacion(frecuencias_estacion = freqs_relativas_anomalias_estacionales_todas_las_estaciones[id_omm][estacion], 
                                                      estacion_del_anio = estacion,
                                                      estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                                      fecha_inicio_str = fecha_inicio_str, 
                                                      fecha_final_str = fecha_final_str, 
                                                      colores =f_paletas_colores.paleta_9_colores,
                                                      incluir_tendencia_ts = True, idioma = "español")


*Periodo en comun con satelite*

In [ ]:
periodo_satelite == True
periodo_completo == False

fecha_inicio_str = "1983-12-15"
fecha_final_str = "2016-11-15"

for id_omm in ides_omm_ordenado:
    for estacion in ["DJF", "MAM", "JJA", "SON"]:
        barplot_oktas_separadas_RELATIVO_por_estacion(frecuencias_estacion = freqs_relativas_anomalias_estacionales_todas_las_estaciones[id_omm][estacion], 
                                                      estacion_del_anio = estacion,
                                                      estaciones = estaciones_ordenadas, id_estacion = id_omm,
                                                      fecha_inicio_str = fecha_inicio_str, 
                                                      fecha_final_str = fecha_final_str, 
                                                      colores =f_paletas_colores.paleta_9_colores,
                                                      incluir_tendencia_ts = True, idioma = "español")


--------------------

**Paso 8: Genero tablas con tendencias**

*Lo hago con las oktas agrupadas en 0, 1-2-3, 4, 5-6-7, 8*

In [ ]:
periodo_satelite == False
periodo_completo == True

fecha_inicio_str = "1961-01-01" 
fecha_final_str = "2020-12-31"

f_tabla_tendencias_agrupo(frecuencias_relativas_anomalias,fecha_inicio_str, fecha_final_str, estaciones_ordenadas, ides_omm_ordenado)

In [ ]:
periodo_satelite == True
periodo_completo == False

fecha_inicio_str = "1983-12-15"
fecha_final_str = "2016-11-15"

f_tabla_tendencias_agrupo(frecuencias_relativas_anomalias,fecha_inicio_str, fecha_final_str, estaciones_ordenadas, ides_omm_ordenado)

*Lo hago por estacion*

In [ ]:
periodo_satelite == False
periodo_completo == True

fecha_inicio_str = "1961-01-01" 
fecha_final_str = "2020-12-31"

f_tabla_tendencias_agrupo_por_estacion(freqs_relativas_anomalias_estacionales_todas_las_estaciones,fecha_inicio_str, fecha_final_str, estaciones_ordenadas, ides_omm_ordenado)

In [ ]:
periodo_satelite == True
periodo_completo == False

fecha_inicio_str = "1983-12-15"
fecha_final_str = "2016-11-15"

f_tabla_tendencias_agrupo_por_estacion(freqs_relativas_anomalias_estacionales_todas_las_estaciones,fecha_inicio_str, fecha_final_str, estaciones_ordenadas, ides_omm_ordenado)

### A PARTIR DE ACA HAGO CALCULOS CON OTRAS VARIABLES

In [ ]:
#%% Abro datos y los acomodo en diccionario
nombres_archivos = os.listdir(ruta_a_archivos)

datos_variables = {}
for nombre_archivo in nombres_archivos:
    data = pd.read_csv(os.path.join(ruta_a_archivos,nombre_archivo), sep="\t", parse_dates = True, na_values = r"\N")     
    data_que_uso = data[["fecha", "tmax", "tmin", "tmed", "prcp"]]
    data_que_uso.index = pd.to_datetime(data_que_uso["fecha"])
    data_que_uso = data_que_uso[["tmax", "tmin", "tmed", "prcp"]]
    data_que_uso.index = pd.to_datetime(data_que_uso.index)
    id_estacion = data["omm_id"].unique()[0]
    datos_variables[id_estacion] = data_que_uso
    


# chequeo errores indiscutibles
# tmax < tmin
ides_omm = list(datos_variables.keys())
for id_omm in ides_omm:
    if sum(datos_variables[id_omm]["tmax"] < datos_variables[id_omm]["tmin"]) != 0:
        print(f"En la estacion {id_omm} exite el caso tmax < tmin")

# pp < 0
for id_omm in ides_omm:
    if sum(datos_variables[id_omm]["prcp"] < 0) != 0:
        print(f"En la estacion {id_omm} exite el caso prcp < 0")

#chequeo outliers de tmax: distancia 3 desvios estandares (mensuales) de la media (mensual)

In [ ]:
*Calculo media mensual de Tmax y Tmin y de PP*

In [ ]:
datos_tmin_tmax_medias_mensuales = {}
datos_pp_acum_mensual = {}

for id_omm in ides_omm:
    datos_tmin_tmax_medias_mensuales[id_omm] = datos_variables[id_omm][["tmax", "tmin"]].groupby(pd.Grouper(freq='M')).mean()
    datos_pp_acum_mensual[id_omm] = datos_variables[id_omm]["prcp"].groupby(pd.Grouper(freq='M')).sum()

*Calculo la climatología (el valor medio del dato mensual) usando el mismo periodo de referencia que para nubosidad*

In [ ]:
#climatologia: periodo comun con satelite (Diciembre 1983 a Noviembre 2016)
fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15"

In [ ]:
tmin_tmax_medias_mensuales_climatologia = {}
pp_acum_mensual_climatologia = {}

for id_omm in ides_omm:
    tmin_tmax_mensuales_fechas_climatologia = datos_tmin_tmax_medias_mensuales[id_omm].loc[fecha_inicio_climatologia_str:fecha_final_climatologia_str]
    tmin_tmax_mensuales_climatologia = tmin_tmax_mensuales_fechas_climatologia.groupby(tmin_tmax_mensuales_fechas_climatologia.index.month).mean()
    tmin_tmax_medias_mensuales_climatologia[id_omm] = tmin_tmax_mensuales_climatologia
    
    pp_mensual_fechas_climatologia = datos_pp_acum_mensual[id_omm].loc[fecha_inicio_climatologia_str:fecha_final_climatologia_str]
    pp_mensual_climatologia = pp_mensual_fechas_climatologia.groupby(pp_mensual_fechas_climatologia.index.month).mean()
    pp_acum_mensual_climatologia[id_omm] = pp_mensual_climatologia

*Calculo anomalias*

In [ ]:
tmin_tmax_medias_mensuales_anomalias = {}
pp_acum_mensual_anomalias = {}

for id_omm in ides_omm:
    tmax_tmin_anomalias_mensuales  = f_serie_anomalias_mensuales(datos_tmin_tmax_medias_mensuales[id_omm], tmin_tmax_medias_mensuales_climatologia[id_omm])
    tmin_tmax_medias_mensuales_anomalias[id_omm] = tmax_tmin_anomalias_mensuales
    pp_anomalias_mensuales  = f_serie_anomalias_mensuales(datos_pp_acum_mensual[id_omm], pp_acum_mensual_climatologia[id_omm])
    pp_acum_mensual_anomalias[id_omm] = pp_anomalias_mensuales

*Calculo tendencias y las acomodo en una tabla*

In [ ]:
def f_tabla_tendencias_tmin_tmax_pp(diccionario_anomalias_tmax_tmin, diccionario_anomalias_pp ,
                                    fecha_inicio_str, fecha_final_str, 
                                    estaciones_ordenadas, ides_omm_ordenado):
    
    numRows = len(ides_omm_ordenado)
    numCols = 3

    tabla_tendencias = pd.DataFrame(index=range(numRows),columns=range(numCols))
    tabla_tendencias_significativas = pd.DataFrame(index=range(numRows),columns=range(numCols))
    
    for j, id_omm in enumerate(ides_omm_ordenado):

        df_anomalias_tmin = diccionario_anomalias_tmax_tmin[id_omm]["tmin"].loc[fecha_inicio_str:fecha_final_str]
        df_anomalias_tmax = diccionario_anomalias_tmax_tmin[id_omm]["tmax"].loc[fecha_inicio_str:fecha_final_str]
        df_anomalias_pp = diccionario_anomalias_pp[id_omm].loc[fecha_inicio_str:fecha_final_str]

        #tmin
        tendencia_tmin = tendencia_theil_sen(df_anomalias_tmin)
        pendiente_tmin = tendencia_tmin[1]*10*12 #decadal
        significativo_tmin = tendencia_tmin[2]
        tabla_tendencias.iloc[j,0] = round(pendiente_tmin, 2) #filas, columnas
        tabla_tendencias_significativas.iloc[j,0] = significativo_tmin
        
        #tmax
        tendencia_tmax = tendencia_theil_sen(df_anomalias_tmax)
        pendiente_tmax = tendencia_tmax[1]*10*12 #decadal
        significativo_tmax = tendencia_tmax[2]
        tabla_tendencias.iloc[j,1] = round(pendiente_tmax, 2) #filas, columnas
        tabla_tendencias_significativas.iloc[j,1] = significativo_tmax

        #pp
        tendencia_pp = tendencia_theil_sen(df_anomalias_pp)
        pendiente_pp = tendencia_pp[1]*10*12 #decadal
        significativo_pp = tendencia_pp[2]
        tabla_tendencias.iloc[j,2] = round(pendiente_pp, 2) #filas, columnas
        tabla_tendencias_significativas.iloc[j,2] = significativo_pp


    tabla_tendencias.index =  list(estaciones_ordenadas.values())
    tabla_tendencias.columns = ["tmin", "tmax", "pp"]
    tabla_tendencias_significativas.index = list(estaciones_ordenadas.values())
    tabla_tendencias_significativas.columns = ["tmin", "tmax", "pp"]
    #guardo
    tabla_tendencias.to_excel(f"tabla_tendencias_tmax_tmin_pp_{fecha_inicio_str}_{fecha_final_str}.xlsx")
    tabla_tendencias_significativas.to_excel(f"tabla_tendencias_significativas_tmax_tmin_pp_{fecha_inicio_str}_{fecha_final_str}.xlsx")
    print(tabla_tendencias)
    print(tabla_tendencias_significativas)
    return(print(f"Se generaron tablas de tendencias de tmax, tmin y pp para el periodo {fecha_inicio_str}_{fecha_final_str}"))


In [ ]:
periodo_satelite = True
periodo_completo = False

fecha_inicio_str = "1983-12-15"
fecha_final_str = "2016-11-15"

#fecha_inicio_str = "1961-01-01" 
#fecha_final_str = "2020-12-31" 

f_tabla_tendencias_tmin_tmax_pp(tmin_tmax_medias_mensuales_anomalias, pp_acum_mensual_anomalias ,
                                fecha_inicio_str, fecha_final_str, 
                                estaciones_ordenadas, ides_omm_ordenado)

In [ ]:
periodo_satelite = False
periodo_completo = True

#fecha_inicio_str = "1983-12-15"
#fecha_final_str = "2016-11-15"

fecha_inicio_str = "1961-01-01" 
fecha_final_str = "2020-12-31" 

f_tabla_tendencias_tmin_tmax_pp(tmin_tmax_medias_mensuales_anomalias, pp_acum_mensual_anomalias ,
                                fecha_inicio_str, fecha_final_str, 
                                estaciones_ordenadas, ides_omm_ordenado)

**Lo hago por estacion**

In [ ]:
freqs_relativas_anomalias_estacionales_todas_las_estaciones = {}

tmin_tmax_medias_mensuales_anomalias_por_estacion = {}
pp_acum_mensual_anomalias_por_estacion = {}


for id_omm in ides_omm_ordenado:
    tmin_medias_mensuales_anomalias_estacionales = {}
    df_data_tmin = tmin_tmax_medias_mensuales_anomalias[id_omm]["tmin"]
    
    tmax_medias_mensuales_anomalias_estacionales = {}
    df_data_tmax = tmin_tmax_medias_mensuales_anomalias[id_omm]["tmax"]
    
    pp_acum_mensual_anomalias_estacionales = {}
    df_data_pp = pp_acum_mensual_anomalias[id_omm]

    for estacion in ["DJF", "MAM", "JJA", "SON"]:
        freq_tmin = []
        freq_tmax = []
        freq_pp = []
        freqs_relativas_anomalias_estacional = [freq_tmin, freq_tmax, freq_pp]
        
        #tmin
        for anio in df_data_tmin.index.year.unique():#iterar sobre los anios

                data = df_data_tmin.loc[str(anio)]

                if estacion == "DJF":
                    if anio == df_data_tmin.index.year[0]:
                        data_estacion = np.nan
                    else:
                        data_anio_previo = df_data_tmin.loc[str(anio-1)]
                        data_estacion = pd.concat([data_anio_previo[data_anio_previo.index.month == 12],
                                                   data[data.index.month ==1],data[data.index.month == 2]])
                        media_estacion = data_estacion.mean(axis = 0)

                elif estacion == "MAM":
                    data_estacion = pd.concat([data[data.index.month == 3],data[data.index.month == 4],data[data.index.month == 5]])
                    media_estacion = data_estacion.mean(axis = 0)

                elif estacion == "JJA":
                    data_estacion = pd.concat([data[data.index.month == 6],data[data.index.month == 7],data[data.index.month == 8]])
                    media_estacion = data_estacion.mean(axis = 0)

                elif estacion == "SON":
                    data_estacion = pd.concat([data[data.index.month == 9],data[data.index.month == 10],data[data.index.month == 11]])
                    media_estacion = data_estacion.mean(axis = 0)

                if type(data_estacion) == pd.DataFrame :
                    freqs_relativas_anomalias_estacional[i].append(media_estacion.loc[str(i)])

        if estacion == "DJF":
            indices = df_data_tmin.index.year.unique()[1: len(df_data_tmin.index.year.unique())] #se saca el primer año
        else:
            indices = df_data_tmin.index.year.unique()


        freqs_relativas_anomalias_estacional_df = pd.DataFrame(freqs_relativas_anomalias_estacional).T
        freqs_relativas_anomalias_estacional_df.index = indices
        freqs_relativas_anomalias_estacionales[estacion] = freqs_relativas_anomalias_estacional_df
    freqs_relativas_anomalias_estacionales_todas_las_estaciones[id_omm] = freqs_relativas_anomalias_estacionales